<a href="https://colab.research.google.com/github/sungkim11/compare-datasets/blob/main/compare_three_datasets_anova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Comparing Datasets: Comparing three datasets (ANOVA or ANalysis Of VAriance)

## 1. Prerequisities

Following are prerequisite for this tutorial:

- Data
- Python Packages: pingouin, statsmodels, and scipy.stats

### 1.1. Data

The dataset used in this notebook was retrieved from kaggle (https://www.kaggle.com/datasets/wordsforthewise/lending-club) and it contains the full Lending Club data available from their site. There are two separate files for accepted and rejected loans. 

For the purpose of this exercise I have used 'fico_range_high' variable from the accepted dataset and 'Risk_Score' variable from the rejected dataset, which I assumed both to be derived credit score of somekind.

For people who would like to understand the data in depth, here is a link of Exploratory Data Analysis (EDA) notebook of dataset here => https://www.kaggle.com/code/wordsforthewise/eda-with-python/notebook.



### 1.2. Python Packages

#### 1.2.1. Install pingouin

Per pingouin's documentation, which is located here => https://pingouin-stats.org/#:

Pingouin is an open-source statistical package written in Python 3 and based mostly on Pandas and NumPy. Some of its main features are listed below. For a full list of available functions, please refer to the API documentation.

- ANOVAs: N-ways, repeated measures, mixed, ancova
- Pairwise post-hocs tests (parametric and non-parametric) and pairwise correlations
- Robust, partial, distance and repeated measures correlations
- Linear/logistic regression and mediation analysis
- Bayes Factors
- Multivariate tests
- Reliability and consistency
- Effect sizes and power analysis
- Parametric/bootstrapped confidence intervals around an effect size or a correlation coefficient
- Circular statistics
- Chi-squared tests
- Plotting: Bland-Altman plot, Q-Q plot, paired plot, robust correlation… italicized text.

In [1]:
%%writefile requirements.txt

pingouin==0.5.2

Writing requirements.txt


In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 185 kB 5.1 MB/s 
     |████████████████████████████████| 9.8 MB 58.0 MB/s 
  Created wheel for pingouin: filename=pingouin-0.5.2-py3-none-any.whl size=196206 sha256=2e0cacaa5ba1b11f0c7e485fc5cf195b3e8df0a18f63715f5685f69928d5e840
  Stored in directory: /root/.cache/pip/wheels/11/5a/63/a6d32fc26fa462c731f65480bfb98ff7bd39b8ebcb4bc6c2fe
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=23216dd913d104c7008f630ca1a9e272c69eeadc636165549277932e3ecb8b0a
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built pingouin littleutils
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


## 2. Code (Three samples from same population)

### 2.1. Import Python Packages

Import python packages and show its version. Showing version is important since it will enable other users to replicate your work using same python version and python packages version.

In [16]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats as stats
import pingouin as pg
import statsmodels as sm
from statsmodels import api
from statsmodels.formula.api import ols

import platform

print('Python: ', platform.python_version())
print('pandas: ', pd.__version__)
print('numpy: ', np.__version__)
print('scipy: ', scipy.__version__)
print('pingouin: ', pg.__version__)
print('statsmodels: ', sm.__version__)

Python:  3.7.15
pandas:  1.3.5
numpy:  1.21.6
scipy:  1.7.3
pingouin:  0.5.2
statsmodels:  0.13.2


### 2.2. Mount Storage

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2.3. Exploratory Data Analysis

#### 2.3.1. Import and validate dataset

In [5]:
accepted_loans = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/stats/data/accepted_2007_to_2018Q4.csv', low_memory=False)
accepted_loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 2.5+ GB


List of variables in accepted_loans dataset.

In [6]:
for col_name in accepted_loans.columns: 
    print(col_name)

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
url
desc
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
fico_range_low
fico_range_high
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
last_fico_range_high
last_fico_range_low
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
to

#### 2.3.2. Cleanse Dataset

Drop all obs where either 'fico_range_low' variable or 'fico_range_high' variable is null.

In [7]:
accepted_loans = accepted_loans.dropna(subset=['fico_range_high', 'fico_range_low'])
accepted_loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2260668 entries, 0 to 2260698
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 2.6+ GB


#### 2.3.3. Downsample Dataset

We do not need a such big dataset and Google colab complains about running out of memory, downsampled the dataset size to 60,000 obs.

In [8]:
accepted_loans = accepted_loans.sample(n=60000)
accepted_loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60000 entries, 140014 to 729857
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 69.6+ MB


#### 2.3.4. Split Dataset

Split the dataset into three datasets to illustrate comparing three datasets with similiar values.

In [9]:
accepted_loans_1s, accepted_loans_2s, accepted_loans_3s = np.array_split(accepted_loans, 3)
accepted_loans_1s.info()
accepted_loans_2s.info()
accepted_loans_3s.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 140014 to 120780
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 23.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 2113278 to 95545
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 23.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 583171 to 729857
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 23.2+ MB


In [10]:
accepted_loans_1s.insert(loc=1, column='dataset', value=1, allow_duplicates=True)
accepted_loans_2s.insert(loc=1, column='dataset', value=2, allow_duplicates=True)
accepted_loans_3s.insert(loc=1, column='dataset', value=3, allow_duplicates=True)

accepted_loans_s = pd.concat([accepted_loans_1s, accepted_loans_2s, accepted_loans_3s])
accepted_loans_s[['id', 'dataset']].groupby('dataset').count()

,id
dataset,
1,20000
2,20000
3,20000


### 2.4. One-way ANOVA using pingouin

In [11]:
aov = pg.anova(dv='fico_range_high', between='dataset', data=accepted_loans_s, detailed=True)

In [12]:
aov.round(3)

,Source,SS,DF,MS,F,p-unc,np2
0,dataset,7.923780e+02,2,396.189,0.364,0.695,0.0
1,Within,6.522043e+07,59997,1087.061,NaN,NaN,NaN


We are considering whether the three samples were drawn from the same population or different populations.

The p-value quantifies the probability of observing as or more extreme values assuming the null hypothesis, that the samples are drawn from populations with the same population means, is true.

Since the 'p value' is greater than 0.05 indicates that the differences are insignificant and our observation is not so unlikely to have occurred by chance. Therefore, we do not reject the null hypothesis of equal population means. This makes sense since the three samples are split from one dataset.

### 2.5. One-way ANOVA using scipy.stats

In [13]:
F, p = stats.f_oneway(accepted_loans_1s['fico_range_high'], accepted_loans_2s['fico_range_high'], accepted_loans_3s['fico_range_high'])
F, p

(0.364458883419413, 0.6945739315676781)

We are considering whether the three samples were drawn from the same population or different populations.

The p-value quantifies the probability of observing as or more extreme values assuming the null hypothesis, that the samples are drawn from populations with the same population means, is true.

Since the 'p value' is greater than 0.05 indicates that the differences are insignificant and our observation is not so unlikely to have occurred by chance. Therefore, we do not reject the null hypothesis of equal population means. This makes sense since the three samples are split from one dataset.

### 2.6. One-way ANOVA using statsmodels

In [14]:
mod = ols('fico_range_high ~ C(dataset)', data=accepted_loans_s).fit()

In [17]:
aov_table = api.stats.anova_lm(mod, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
C(dataset),7.923784e+02,2.0,0.364459,0.694574
Residual,6.522043e+07,59997.0,NaN,NaN


We are considering whether the three samples were drawn from the same population or different populations.

The p-value quantifies the probability of observing as or more extreme values assuming the null hypothesis, that the samples are drawn from populations with the same population means, is true.

Since the 'p value' is greater than 0.05 indicates that the differences are insignificant and our observation is not so unlikely to have occurred by chance. Therefore, we do not reject the null hypothesis of equal population means. This makes sense since the three samples are split from one dataset.

## 3. Code (Three samples from different population)

### 3.1. Exploratory Data Analysis

#### 3.1.1. Import and validate dataset

In [18]:
rejected_loans = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/stats/data/rejected_2007_to_2018Q4.csv', low_memory=False)
rejected_loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27648741 entries, 0 to 27648740
Data columns (total 9 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount Requested      float64
 1   Application Date      object 
 2   Loan Title            object 
 3   Risk_Score            float64
 4   Debt-To-Income Ratio  object 
 5   Zip Code              object 
 6   State                 object 
 7   Employment Length     object 
 8   Policy Code           float64
dtypes: float64(3), object(6)
memory usage: 1.9+ GB


#### 3.1.2. Cleanse Dataset

Drop all obs where 'Risk_Score' variable is null.

In [19]:
rejected_loans = rejected_loans.dropna(subset=['Risk_Score'])
rejected_loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9151111 entries, 0 to 27648740
Data columns (total 9 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount Requested      float64
 1   Application Date      object 
 2   Loan Title            object 
 3   Risk_Score            float64
 4   Debt-To-Income Ratio  object 
 5   Zip Code              object 
 6   State                 object 
 7   Employment Length     object 
 8   Policy Code           float64
dtypes: float64(3), object(6)
memory usage: 698.2+ MB


#### 3.1.3. Downsample Dataset

We only need to replace one dataset, the dataset downsampled to 20,000 obs.

In [20]:
rejected_loans = rejected_loans.sample(n=20000)
rejected_loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 591783 to 25664822
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Amount Requested      20000 non-null  float64
 1   Application Date      20000 non-null  object 
 2   Loan Title            20000 non-null  object 
 3   Risk_Score            20000 non-null  float64
 4   Debt-To-Income Ratio  20000 non-null  object 
 5   Zip Code              20000 non-null  object 
 6   State                 20000 non-null  object 
 7   Employment Length     19671 non-null  object 
 8   Policy Code           20000 non-null  float64
dtypes: float64(3), object(6)
memory usage: 1.5+ MB


#### 3.1.4. Split Accepted Loans Dataset

In [21]:
accepted_loans_1d, accepted_loans_2d, accepted_loans_3d = np.array_split(accepted_loans, 3)
accepted_loans_1d.info()
accepted_loans_2d.info()
accepted_loans_3d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 140014 to 120780
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 23.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 2113278 to 95545
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 23.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 583171 to 729857
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 23.2+ MB


#### 3.1.5. Replace accepted_loans_3d dataset with values from rejected_loans dataset.

In [22]:
accepted_loans_3d['fico_range_high'].mean()

702.5884

In [23]:
accepted_loans_3d['fico_range_high'] = np.where(accepted_loans_3d['fico_range_high']>0, rejected_loans['Risk_Score'], rejected_loans['Risk_Score'])

In [24]:
accepted_loans_3d['fico_range_high'].mean()

628.4756

#### 3.1.6. Re-join the dataset

In [25]:
accepted_loans_1d.insert(loc=1, column='dataset', value=1, allow_duplicates=True)
accepted_loans_2d.insert(loc=1, column='dataset', value=2, allow_duplicates=True)
accepted_loans_3d.insert(loc=1, column='dataset', value=3, allow_duplicates=True)

accepted_loans_d = pd.concat([accepted_loans_1d, accepted_loans_2d, accepted_loans_3d])
accepted_loans_d[['id', 'dataset']].groupby('dataset').count()

,id
dataset,
1,20000
2,20000
3,20000


### 3.2. One-way ANOVA using pingouin

In [26]:
aov = pg.anova(dv='fico_range_high', between='dataset', data=accepted_loans_d, detailed=True)

In [27]:
aov.round(3)

,Source,SS,DF,MS,F,p-unc,np2
0,dataset,7.333536e+07,2,3.666768e+07,10759.229,0.0,0.264
1,Within,2.044710e+08,59997,3.408021e+03,NaN,NaN,NaN


We are considering whether the three samples were drawn from the same population or different populations.

The p-value quantifies the probability of observing as or more extreme values assuming the null hypothesis, that the samples are drawn from populations with the same population means, is true.

Since the 'p value' is less than 0.05 indicates that the differences are significant. Therefore, we reject the null hypothesis of equal population means. This makes sense since the three samples are derived from different datasets.

### 3.3. One-way ANOVA using scipy.stats

In [28]:
F, p = stats.f_oneway(accepted_loans_1d['fico_range_high'], accepted_loans_2d['fico_range_high'], accepted_loans_3d['fico_range_high'])
F, p

(10759.228812563964, 0.0)

We are considering whether the three samples were drawn from the same population or different populations.

The p-value quantifies the probability of observing as or more extreme values assuming the null hypothesis, that the samples are drawn from populations with the same population means, is true.

Since the 'p value' is less than 0.05 indicates that the differences are significant. Therefore, we reject the null hypothesis of equal population means. This makes sense since the three samples are derived from different datasets.

### 3.4. One-way ANOVA using statsmodels

In [29]:
mod = ols('fico_range_high ~ C(dataset)', data=accepted_loans_d).fit()

In [30]:
aov_table = api.stats.anova_lm(mod, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
C(dataset),7.333536e+07,2.0,10759.228813,0.0
Residual,2.044710e+08,59997.0,NaN,NaN


We are considering whether the three samples were drawn from the same population or different populations.

The p-value quantifies the probability of observing as or more extreme values assuming the null hypothesis, that the samples are drawn from populations with the same population means, is true.

Since the 'p value' is less than 0.05 indicates that the differences are significant. Therefore, we reject the null hypothesis of equal population means. This makes sense since the three samples are derived from different datasets.